<a href="https://colab.research.google.com/github/Ev3lynJim3nezvelez/hola-mundo-phyton/blob/main/iaupb_solucion_examen_02_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Etapa 1: lectura y procesamiento inicial**

In [ ]:
import pandas as pd
print("carga y primera inspeccion")
#cargar los datos

df = pd.read_csv('registro_llegadas_a_medellin_dic2024.csv', sep= ';', encoding= 'utf-8')
print(f"dataset cargado exitosamente")
print(f"Dimensiones del dataset: {df.shape[0]} filas {df.shape[1]} columnas")

#Informacion basica del datset

print("\nPrimeras 5 filas:")
print(df.head())

print("\nultimas 5 filas:")
print(df.tail())

print("\nTipos de datos:")
print(df.dtypes)

print("\nNombres de columnas:")
print(df.colums.tolist())




In [ ]:
#duplicados
print("\nRegistros duplicados:")

total_duplicados = df.duplicated().sum()

if total_duplicados ==0:
    print("No se encontraron registros duplicados")
else:
    print(f"\nTotal registros duplicados encontrados : {total_duplicados} ({(total_duplicados/len(df)*100):.2f}%) \n")

    duplicados = df.groupby(['FechaObservacion', 'ValorObservado']).size().reset_index(name='veces_duplicado')
    # Filtra solo los registros que están duplicados (aparecen más de una vez)
    duplicados = duplicados[duplicados['veces_duplicado'] > 1]
    print(duplicados)

#conversion fecha
print(f"\nConversión de fechas:")
try:
    df_procesado['FechaObservacion'] = pd.to_datetime(df_procesado['FechaObservacion'])
    print(" Conversión de fechas exitosa")

except Exception as e:
    print(f"Error en conversión de fechas: Posible inconsistencia de formato\n")

try:
    print('Intentando conversión de fechas con formato mezclado... un momento por favor!')

    df_procesado['FechaObservacion'] = pd.to_datetime(df_procesado['FechaObservacion'], format='mixed')
    print("Conversión de fechas exitosa")


    fecha_min = df_procesado['FechaObservacion'].min()
    fecha_max = df_procesado['FechaObservacion'].max()
    duracion = fecha_max - fecha_min

    print(f" Rango temporal: {fecha_min} a {fecha_max}")
    print(f" Duración total: {duracion.days} días, {duracion.seconds//3600} horas")

except Exception as e:
    print(f"Error en conversión de fechas: {e}")



**Etapa 2: identificar outliers sin eliminarlos**

In [ ]:
#deteccion de outliers

Q1 = df['ValorObservado'].quantile(0.25)
Q3 = df['ValorObservado'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers = df[(df['ValorObservado'] < limite_inferior) | (df['ValorObservado'] > limite_superior)]

print(f"\nDetección de outliers:")
print(f"  - Límite inferior: {limite_inferior:.2f}")
print(f"  - Límite superior: {limite_superior:.2f}")
print(f"  - Outliers detectados: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")

**Etapa 3: Identificar comportamiento por hora y dia de la semana**

In [ ]:
# Crear características temporales
df_temporal = df_procesado.copy()
df_temporal['hora'] = df_temporal['FechaObservacion'].dt.hour
df_temporal['dia_semana'] = df_temporal['FechaObservacion'].dt.dayofweek  # 0=Lunes, 6=Domingo
df_temporal['dia_semana_nombre'] = df_temporal['FechaObservacion'].dt.day_name()
df_temporal['semana_año'] = df_temporal['FechaObservacion'].dt.isocalendar().week
df_temporal['mes'] = df_temporal['FechaObservacion'].dt.month
df_temporal['mes_nombre'] = df_temporal['FechaObservacion'].dt.month_name()
df_temporal['fecha'] = df_temporal['FechaObservacion'].dt.date

# Crear orden correcto de días de la semana (en español)
dias_semana_orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dias_semana_es = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

# Crear mapeo de los días de inglés a español para mejor visualización
mapeo_dias = dict(zip(dias_semana_orden, dias_semana_es))
df_temporal['dia_semana_nombre_es'] = df_temporal['dia_semana_nombre'].map(mapeo_dias)

# Crear orden correcto de los meses del año (en español)
meses_orden = ['January', 'February', 'March', 'April', 'May', 'June', 'July','August', 'September', 'October', 'November','December']
meses_es = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio','Agosto', 'Septiembre', 'Octubre', 'Noviembre','Diciembre']

# Crear mapeo de los meses de inglés a español para mejor visualización
mapeo_meses = dict(zip(meses_orden, meses_es))
df_temporal['mes_nombre_es'] = df_temporal['mes_nombre'].map(mapeo_meses)

# Crear variables categóricas ordenadas - Solo español
df_temporal['DiaSemanaOrdenadoEs'] = pd.Categorical(df_temporal['dia_semana_nombre_es'], categories=dias_semana_es, ordered=True)
df_temporal['MesOrdenadoEs'] = pd.Categorical(df_temporal['mes_nombre_es'], categories=meses_es, ordered=True)

print("Características temporales extraídas: hora, día_semana, semana_año, mes")

# Análisis por hora del día
print(f"\n--- Análisis por Hora del Día ---")
comp_por_hora = df_temporal.groupby('hora')['ValorObservado'].agg([
    'count', 'mean', 'std', 'min', 'max'
]).round(2)
comp_por_hora.columns = ['Mediciones', 'Promedio', 'Desv_Std', 'Mínimo', 'Máximo']

print("Estadísticas de comportamiento por hora:")
print(comp_por_hora)

# Horas con mayor y menor temperatura promedio
hora_max_comp = comp_por_hora['Promedio'].idxmax()
hora_min_comp = comp_por_hora['Promedio'].idxmin()
print(f"\nHora con mayor comportamiento promedio: {hora_max_comp}:00 ({comp_por_hora.loc[hora_max_comp, 'Promedio']:.2f})")
print(f"Hora con menor comportamiento promedio: {hora_min_comp}:00 ({comp_por_hora.loc[hora_min_comp, 'Promedio']:.2f})")
print(f"Variación diaria: {comp_por_hora['Promedio'].max() - comp_por_hora['Promedio'].min():.2f}°C")

# Análisis por dia de la semana
print(f"\n--- Análisis por Día de la Semana ---")
comp_por_dia = df_temporal.groupby(['dia_semana', 'dia_semana_nombre_es'])['ValorObservado'].agg([
    'count', 'mean', 'std', 'min', 'max'
]).round(2)
comp_por_dia.columns = ['Mediciones', 'Promedio', 'Desv_Std', 'Mínimo', 'Máximo']

print("Estadísticas de temperatura por día de la semana:")
print(comp_por_dia)

# Día con mayor y menor temperatura promedio
dia_max_temp = comp_por_dia['Promedio'].idxmax()
dia_min_temp = comp_por_dia['Promedio'].idxmin()
print(f"\nDía con mayor temperatura promedio: {dia_max_temp[1]} ({comp_por_dia.loc[dia_max_temp, 'Promedio']:.2f})")
print(f"Día con menor temperatura promedio: {dia_min_temp[1]} ({comp_por_dia.loc[dia_min_temp, 'Promedio']:.2f})")


**Etapa 4: visualizaciones**

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2,2, figsize =(20,15))

comp_hora = comp_por_hora['promedio']
axes[0,0].plot(comp_hora.index, comp_hora.values, marker='o', linewidth=2, markersize=6)
axes[0,0].set_title('Comportamiento promedio por Hora del Día', fontsize=12, fontweight='bold')
axes[0,0].set_xlabel('Hora del día')
axes[0,0].set_ylabel('comportamiento')
axes[0,0].set_xticks(range(0, 24, 4))
axes[0,0].grid(True, alpha=0.3)
